In [ ]:
# Download arabic wikipedia extract

# Extract it with WikiExtractor

# Romanize it with uroman

In [ ]:
'''
import contextlib
import hashlib
import itertools
import os
import shutil
import sys
import time
from collections import namedtuple
from pathlib import Path

import filehash
import fire
import numpy
import pandas as pd
import pendulum
import sarge
import torch
'''

In [8]:
import logging as logger 

In [ ]:
def _download_file_maybe(url: str, out_file_path=None):
    """NOTE: FUNCTION NOT WORKING.
    
    Download a file unless a file already exists at destination path. In all
    cases, log a digest with checksum of the file that is present."""
    if out_file_path is None:
        out_file_path = Path('.') / Path(Path(url).name)

    if out_file_path.is_file():
        logger.warning(
            "Not downloading because there is already a file at: {} ({})".format(
                out_file_path, out_file_path.absolute()
            )
        )
    else:
        logger.info(
            "Now we are going to download the COHA sample corpus from the internet"
        )
        # https://stackoverflow.com/questions/1078524/how-to-specify-the-download-location-with-wget
        command = sarge.shell_format("wget -q -O {} {} ", out_file_path, url)
        # logger.info(f'Running {command=}')
        _run_cmd(command, cmd_label="wget")

    logger.debug(f"{file_info_digest(out_file_path)=}")

## Download arabic wikipedia extract

In [4]:
# NO! # https://dumps.wikimedia.org/arwiki/latest/arwiki-latest-pages-articles.xml.bz2 
# YES # https://dumps.wikimedia.org/arwiki/20230201/arwiki-20230201-pages-articles.xml.bz2  

# https://dumps.wikimedia.org/arwiki/20230201/arwiki-20230201-sha1sums.txt 

'''
https://dumps.wikimedia.org/arwiki/20230201/

See instructions here: https://github.com/attardi/wikiextractor
'''

'\nhttps://dumps.wikimedia.org/arwiki/20230201/\n\nSee instructions here: https://github.com/attardi/wikiextractor\n'

In [3]:
# ! wget https://dumps.wikimedia.org/arwiki/20230201/arwiki-20230201-sha1sums.txt 

fish: Unknown command: fish_add_path
/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish (line 34): 
fish_add_path --prepend --global "/nix/var/nix/profiles/default/bin"
^
from sourcing file /nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish
	called on line 185 of file /usr/share/fish/config.fish
in function '.' with arguments '/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish'
	called on line 4 of file /etc/fish/conf.d/nix.fish
from sourcing file /etc/fish/conf.d/nix.fish
	called on line 294 of file /usr/share/fish/config.fish
from sourcing file /usr/share/fish/config.fish
	called during startup
fish: Unknown command: fish_add_path
/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish (line 35): 
fish_add_path --prepend --global "$HOME/.nix-profile/bin"
^
from sourcing file /nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish
	called on line 185 of file /usr/share/fish/config.fish
in function '.' with arguments '/nix/var/nix/profiles/default/etc

In [2]:
#! wget https://dumps.wikimedia.org/arwiki/20230201/arwiki-20230201-pages-articles.xml.bz2

fish: Unknown command: fish_add_path
/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish (line 34): 
fish_add_path --prepend --global "/nix/var/nix/profiles/default/bin"
^
from sourcing file /nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish
	called on line 185 of file /usr/share/fish/config.fish
in function '.' with arguments '/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish'
	called on line 4 of file /etc/fish/conf.d/nix.fish
from sourcing file /etc/fish/conf.d/nix.fish
	called on line 294 of file /usr/share/fish/config.fish
from sourcing file /usr/share/fish/config.fish
	called during startup
fish: Unknown command: fish_add_path
/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish (line 35): 
fish_add_path --prepend --global "$HOME/.nix-profile/bin"
^
from sourcing file /nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish
	called on line 185 of file /usr/share/fish/config.fish
in function '.' with arguments '/nix/var/nix/profiles/default/etc

In [5]:
! sha1sum --ignore-missing --check arwiki-20230201-sha1sums.txt  

fish: Unknown command: fish_add_path
/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish (line 34): 
fish_add_path --prepend --global "/nix/var/nix/profiles/default/bin"
^
from sourcing file /nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish
	called on line 185 of file /usr/share/fish/config.fish
in function '.' with arguments '/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish'
	called on line 4 of file /etc/fish/conf.d/nix.fish
from sourcing file /etc/fish/conf.d/nix.fish
	called on line 294 of file /usr/share/fish/config.fish
from sourcing file /usr/share/fish/config.fish
	called during startup
fish: Unknown command: fish_add_path
/nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish (line 35): 
fish_add_path --prepend --global "$HOME/.nix-profile/bin"
^
from sourcing file /nix/var/nix/profiles/default/etc/profile.d/nix-daemon.fish
	called on line 185 of file /usr/share/fish/config.fish
in function '.' with arguments '/nix/var/nix/profiles/default/etc

## Extract it with WikiExtractor

In [6]:
'''
NOTE: wikiextractor will not work with python 3.11. It will work with 3.10.

Option --no-templates significantly speeds up the extractor, avoiding the cost of expanding MediaWiki templates.
'''
# ! time python -m wikiextractor.WikiExtractor arwiki-20230201-pages-articles.xml.bz2 

'''
With template processing included:

...
INFO: Preprocessed 3400000 pages
INFO: Loaded 126172 templates in 425.4s
'''

# ! time python -m wikiextractor.WikiExtractor --no-templates arwiki-20230201-pages-articles.xml.bz2   

'''With --no-templates, we jump immediately to the extraction step (no preprocessing pages).'''

'\nWith template procesing included:\n\n...\nINFO: Preprocessed 3400000 pages\nINFO: Loaded 126172 templates in 425.4s\n'

```bash
╭─user at s-m2tal in ⌁/cours/LexResCerisara/ANSProj/faroese_mBERT (scott ●3…104)
╰─λ time python -m wikiextractor.WikiExtractor --no-templates arwiki-20230201-pages-articles.xml.bz2                                    (LexResProject) 0 (0.001s) < 16:59:17
INFO: Starting page extraction from arwiki-20230201-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Extracted 100000 articles (1094.8 art/s)
INFO: Extracted 200000 articles (1673.1 art/s)
INFO: Extracted 300000 articles (1844.7 art/s)
INFO: Extracted 400000 articles (1650.6 art/s)
INFO: Extracted 500000 articles (1812.1 art/s)
INFO: Extracted 600000 articles (1985.7 art/s)
INFO: Extracted 700000 articles (3096.2 art/s)
INFO: Extracted 800000 articles (2146.3 art/s)
INFO: Extracted 900000 articles (2016.3 art/s)
INFO: Extracted 1000000 articles (1978.5 art/s)
INFO: Extracted 1100000 articles (1490.3 art/s)
INFO: Extracted 1200000 articles (2329.1 art/s)
INFO: Extracted 1300000 articles (2531.2 art/s)
INFO: Extracted 1400000 articles (2942.8 art/s)
INFO: Extracted 1500000 articles (3129.7 art/s)
INFO: Extracted 1600000 articles (2685.5 art/s)
INFO: Extracted 1700000 articles (3340.0 art/s)
INFO: Extracted 1800000 articles (2942.4 art/s)
INFO: Extracted 1900000 articles (1992.6 art/s)
INFO: Extracted 2000000 articles (1589.1 art/s)
INFO: Extracted 2100000 articles (1736.0 art/s)
INFO: Extracted 2200000 articles (1526.2 art/s)
INFO: Finished 1-process extraction of 2219910 articles in 1115.8s (1989.6 art/s)

________________________________________________________
Executed in   18.60 mins   fish           external 
   usr time  1541.75 secs  357.00 micros  1541.75 secs 
   sys time   87.66 secs  135.00 micros   87.66 secs 

╭─user at s-m2tal in ⌁/cours/LexResCerisara/ANSProj/faroese_mBERT (scott ●3✚1…2261)
╰─λ                                                                                                                                  (LexResProject) 0 (18:35.847) < 17:18:1100000
    
```

https://github.com/attardi/wikiextractor/wiki/File-Format

"
Document files contains a series of Wikipedia articles, represented each by an XML <tt>doc</tt> element: ... ... ... ...

The element <tt>doc</tt> has the following attributes:

    <tt>id</tt>, which identifies the document by means of a unique serial number
    <tt>url</tt>, which provides the URL of the original Wikipedia page.

The content of a <tt>doc</tt> element consists of pure text, one sentence per line.

Here is an example of a <tt>doc</tt> element:
Harmonium. L'harmonium è uno strumento musicale azionato con una tastiera, detta manuale. Sono stati costruiti anche alcuni harmonium con due manuali. ...

Notice that because of Wikipedia conventions, the first sentence is the title of the article.

Such documents are produced by Wikipedia Extractor .
"

## Romanize it with uroman

Cf. https://github.com/isi-nlp/uroman 

...

### Usage
```bash
$ uroman.pl [-l <lang-code>] [--chart] [--no-cache] < STDIN
       where the optional <lang-code> is a 3-letter languages code, e.g. ara, bel, bul, deu, ell, eng, fas,
            grc, ell, eng, heb, kaz, kir, lav, lit, mkd, mkd2, oss, pnt, pus, rus, srp, srp2, tur, uig, ukr, yid.
       --chart specifies chart output (in JSON format) to represent alternative romanizations.
       --no-cache disables caching.
```
### Examples
```bash
$ bin/uroman.pl < text/zho.txt
$ bin/uroman.pl -l tur < text/tur.txt
$ bin/uroman.pl -l heb --chart < text/heb.txt
$ bin/uroman.pl < test/multi-script.txt > test/multi-script.uroman.txt
```

Identifying the input as Arabic, Belarusian, Bulgarian, English, Farsi, German,
Ancient Greek, Modern Greek, Pontic Greek, Hebrew, Kazakh, Kyrgyz, Latvian,
Lithuanian, North Macedonian, Russian, Serbian, Turkish, Ukrainian, Uyghur or 
Yiddish will improve romanization for those languages as some letters in those 
languages have different sound values from other languages using the same script 
(French, Russian, Hebrew respectively).
No effect for other languages in this version.

...

script uroman-quick.pl for Arabic script languages, incl. Uyghur.
   Much faster, pre-caching mapping of Arabic to Latin characters, simple greedy processing.


In [ ]:
# ! uroman.pl -l ara   < STDIN   > BLA.uroman.txt

# !  ./uroman.pl -l ara   < ~/cours/LexResCerisara/ANSProj/faroese_mBERT/text/AA/wiki_00   | less   

```bash
╭─user at s-m2tal in ⌁/cours/LexResCerisara/ANSProj/uroman/bin (master ✔)
╰─λ ./uroman.pl -l ara   < ~/cours/LexResCerisara/ANSProj/faroese_mBERT/text/AA/wiki_00   | tail                     

tkhtlf aljam'a al'rbya akhtlafa kbyra 'n alathad alawrwby, flm thqq aljam'a al'rbya mqdara mlhwza mn altkaml aliqlymy, wlys lljam'a 'laqa mbashra m' aldwl ala'da'. wlkn aljam'a al'rbya mbnya 'la mbadye td'm wtrwj lqwmya 'rbya mwhda wtwhyd mwaqf aldwl ala'da' bkhsws mkhtlf alqdaya.
jmy' a'da' aljam'a al'rbya a'da' fy mnzma almwtmr alislamy. kma an hnak mjmw'at fr'ya fy aljam'a, mthl «mjls alt'awn alkhlyjy» w«athad almghrb al'rby».
qayema alqmm al'rbya.
mnth tasys aljam'a al'rbya fy 'am 1945 'qd alqada al'rb 39 ajtma' qma, bynha 25 qma 'adya w11 qmm taryea, ila janb thlatha qmm aqtsadya:
alantqadat.
in jam'a aldwl al'rbya lys fqt twajh alantqadat mn aldwl alakhra
bl inha lys lha sh'bya lda m'zm alsh'wb al'rbya bsbb anha fshlt fy hl alkthyr mn alqdaya al'rbya wl'l abrzha alsra' al'rby alisrayeyly wahtlal alkyan alshywny llarady alflstynya 'am 1948 w'am 1967
walkthyr mn alqdaya kalazma alswrya whl mshkla alsh'b alswry althy asbh y'any alljw' waltshryd fy aldwl almjawra walghyr mjawra 'da anh ywajh almwt mn jmy' atraf alnza' dakhl wtnh swrya.

</doc>

╭─user at s-m2tal in ⌁/cours/LexResCerisara/ANSProj/uroman/bin (master ✔)
                                                                                     
(base) 0 (10.730s) < 17:26:52

```


```bash
╭─user at s-m2tal in ⌁/cours/LexResCerisara/ANSProj/uroman/bin (master ✔)
╰─λ ./uroman-quick.pl -l ara   < ~/cours/LexResCerisara/ANSProj/faroese_mBERT/text/AA/wiki_00   | tail               

..
tkhtlf aljam'a al'rbya akhtlafa kbyra 'n alathad alawrwby, flm thqq aljam'a al'rbya mqdara mlhwza mn altkaml aliqlymy, wlys lljam'a 'laqa mbashra m' aldwl ala'da'. wlkn aljam'a al'rbya mbnya 'la mbadye td'm wtrwj lqwmya 'rbya mwhda wtwhyd mwaqf aldwl ala'da' bkhsws mkhtlf alqdaya.
jmy' a'da' aljam'a al'rbya a'da' fy mnzma almwtmr alislamy. kma an hnak mjmw'at fr'ya fy aljam'a, mthl «mjls alt'awn alkhlyjy» w«athad almghrb al'rby».
qayema alqmm al'rbya.
mnth tasys aljam'a al'rbya fy 'am 1945 'qd alqada al'rb 39 ajtma' qma, bynha 25 qma 'adya w11 qmm taryea, ila janb thlatha qmm aqtsadya:
alantqadat.
in jam'a aldwl al'rbya lys fqt twajh alantqadat mn aldwl alakhra
bl inha lys lha sh'bya lda m'zm alsh'wb al'rbya bsbb anha fshlt fy hl alkthyr mn alqdaya al'rbya wl'l abrzha alsra' al'rby alisrayeyly wahtlal alkyan alshywny llarady alflstynya 'am 1948 w'am 1967
walkthyr mn alqdaya kalazma alswrya whl mshkla alsh'b alswry althy asbh y'any alljw' waltshryd fy aldwl almjawra walghyr mjawra 'da anh ywajh almwt mn jmy' atraf alnza' dakhl wtnh swrya.

</doc>
╭─user at s-m2tal in ⌁/cours/LexResCerisara/ANSProj/uroman/bin (master ✔)
╰─λ                                                                                                                 

(base) 0 (3.287s) < 17:29:39
```


This output so far looks pretty good. Remaining tasks to do:

- remove wikitext code like `[[` etc
- for extractfile in files: for doc in extractfile: append doc to big concat file of full corpus
- *maybe* also filter corpus (e.g. X number of docs from each file; randomly select which docs)
- Q: should we run uroman on the full dataset after all wikitext stripping and concat; 
    or on each doc as we feed it in?; or at some other point in pipeline?
